In [44]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import pickle , joblib
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [23]:
x , y = [], []
with open("SMSSpamcollection.txt","r")as file:
    data = file.read()
    for data_item in data.split("\n"):
        if len(data_item.split("\t")) == 2:
            
            x.append(data_item.split("\t")[1])

            y.append(data_item.split("\t")[0])
print("success")


success


In [24]:
len (x),len(y)

(10752, 10752)

In [25]:
import pandas as pd  # Make sure this is at the top

data_dict = {"mail_message": x, "label": y}
df = pd.DataFrame(data_dict)
df.head()


,mail_message,label
0,You have 1 new voicemail. Call 0945060571 to l...,spam
1,You have 1 new voicemail. Call 0946959784 to l...,spam
2,You've won 2 tickets to the Bahamas! Call now ...,spam
3,URGENT! You have won a 1 week FREE membership ...,spam
4,Claim your FREE RINGTONE now by texting FREE t...,spam


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10752 entries, 0 to 10751
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   mail_message  10752 non-null  object
 1   label         10752 non-null  object
dtypes: object(2)
memory usage: 168.1+ KB


In [27]:
df.isnull().sum()

mail_message    0
label           0
dtype: int64

In [28]:
import re
for i in range(0,len(df)):
    message = re.sub("[^a-zA-Z]"," ",df["mail_message"][i])
    message = message.lower()
    df["mail_message"][i] = message
df.head()    

C:\Users\RAMAVTAR\AppData\Local\Temp\ipykernel_10432\761683756.py:5: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df["mail_message"][i] = message


,mail_message,label
0,you have new voicemail call to l...,spam
1,you have new voicemail call to l...,spam
2,you ve won tickets to the bahamas call now ...,spam
3,urgent you have won a week free membership ...,spam
4,claim your free ringtone now by texting free t...,spam


In [29]:
df["label"].value_counts()

label
spam    5925
ham     4827
Name: count, dtype: int64

In [30]:
df["label"] = df["label"].map({"ham":0,"spam":1})
df.head()


,mail_message,label
0,you have new voicemail call to l...,1
1,you have new voicemail call to l...,1
2,you ve won tickets to the bahamas call now ...,1
3,urgent you have won a week free membership ...,1
4,claim your free ringtone now by texting free t...,1


In [31]:
df.isnull().sum()
df.drop_duplicates(inplace=True)

In [32]:
x = df["mail_message"]
y = df["label"]


In [33]:
#PREPARE THE SINGLE LIST OF MAIL
X = x.values

In [34]:
x[0:5]

0    you have   new voicemail  call            to l...
2    you ve won   tickets to the bahamas  call now ...
3    urgent  you have won a   week free membership ...
4    claim your free ringtone now by texting free t...
5    freemsg  you ve been selected to receive a fre...
Name: mail_message, dtype: object

In [35]:
##bag of words
Bow_obj = CountVectorizer(
    max_features=2000,
    stop_words =("english")
)
##into array -- sparse matrix -- dense matrix
x_transformed = Bow_obj.fit_transform(x).toarray()
x_transformed.shape

(6214, 2000)

In [36]:
joblib.dump(Bow_obj,"Vectorizer.lb")

['Vectorizer.lb']

In [37]:
#train test split
x_train, x_text, y_train, y_text = train_test_split(
    x_transformed ,y ,
    test_size= 0.2
)

In [38]:
BNB= BernoulliNB()
BNB.fit(x_train,y_train)

,alpha,1.0
,force_alpha,True
,binarize,0.0
,fit_prior,True
,class_prior,None


In [39]:
BNB.score(x_text,y_text)
BNB.score(x_train,y_train)



0.9798833232749949

In [40]:
pred = BNB.predict(x_text)
pred

array([1, 1, 0, ..., 0, 0, 0], shape=(1243,))

In [41]:
compare_dict = {"Actual":y_text,"Predicted":pred   }
compare_df = pd.DataFrame(compare_dict)
compare_df.head()

,Actual,Predicted
7969,1,1
5993,1,1
9491,0,0
7202,0,0
6764,0,0


In [42]:
pickle.dump(BNB,open("spamclassifiermodel.pkl","wb"))